In [1]:
import pandas as pd


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# read in the COMPLETE dataset, sort it by date, and turn date column to pythonic DateTime
df = pd.read_csv('../data/CPD_2001_to_present.csv', parse_dates= ['Date'])
df = df.sort_values(by= ['Date'])
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))

/Users/raypaller/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### The date-stratified CPD data is now read in its entirety and date sorted;  Use the last 7 years for analysis

In [3]:
df = df[df['Date'] > pd.to_datetime('2012-11-15')]
df.shape

(1918909, 22)

In [4]:
df.head(3)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6369822,8926266,HV598372,2012-11-15 00:01:00,013XX W CORNELIA AVE,0810,THEFT,OVER $500,APARTMENT,False,False,1924,19.0,44.0,6.0,06,1166701.0,1923359.0,2012,02/10/2018 03:50:01 PM,41.945267,-87.662672,"(41.945267201, -87.662671562)"
6346887,8888158,HV562478,2012-11-15 00:01:00,122XX S PRINCETON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,523,5.0,34.0,53.0,14,1176488.0,1823849.0,2012,02/10/2018 03:50:01 PM,41.671987,-87.629693,"(41.67198685, -87.629693199)"
6346904,8888176,HV562494,2012-11-15 00:01:00,012XX N PULASKI RD,0460,BATTERY,SIMPLE,SIDEWALK,False,False,2534,25.0,27.0,23.0,08B,1149498.0,1907791.0,2012,02/10/2018 03:50:01 PM,41.902899,-87.726309,"(41.902899152, -87.72630861)"


#### District names and district numbers where occurred to be had via the 'beat' number in the crime report

In [5]:
df_beat_locations = pd.read_csv('../data/CPD_beat_locations.csv')

df_cpd_by_neighborhood = pd.read_csv('../data/CPD_districts_by_neighborhood.csv')
df_cpd_by_neighborhood = df_cpd_by_neighborhood.dropna()
df_cpd_by_neighborhood.columns = ['district', 'district_name']
df_cpd_by_neighborhood = df_cpd_by_neighborhood.reset_index().drop(['index'], axis= 1)
df_cpd_by_neighborhood

,district,district_name
0,1.0,Central
1,2.0,Wentworth
2,3.0,Grand Crossing
3,4.0,South Chicago
4,5.0,Calumet
5,6.0,Gresham
6,7.0,Englewood
7,8.0,Chiocago Lawn
8,9.0,Deering
9,10.0,Ogden


In [6]:
df_beat_locations.head()

,the_geom,DISTRICT,SECTOR,BEAT,BEAT_NUM
0,MULTIPOLYGON (((-87.7047252651434 41.975774430...,17,1,1,1713
1,MULTIPOLYGON (((-87.83365455041093 41.97535481...,31,0,0,3100
2,MULTIPOLYGON (((-87.90684167275818 41.97656175...,16,5,5,1651
3,MULTIPOLYGON (((-87.64491798475646 41.96972709...,19,1,1,1914
4,MULTIPOLYGON (((-87.63724132684592 41.96598776...,19,1,1,1915


#### Drop the geometry and goto lower caps

In [7]:
df_beat_locations = df_beat_locations.drop(['the_geom'], axis= 1)
df_beat_locations.columns = ['district', 'sector', 'beat', 'beat_num']

In [8]:
df_beat_by_district_neighborhood_name = pd.merge(df_beat_locations, df_cpd_by_neighborhood,\
                                                 how= 'left', left_on= 'district', right_on= 'district')
df_beat_by_district_neighborhood_name.head()

,district,sector,beat,beat_num,district_name
0,17,1,1,1713,Albany Park
1,31,0,0,3100,NaN
2,16,5,5,1651,Jefferson Park
3,19,1,1,1914,Town Hall
4,19,1,1,1915,Town Hall


In [9]:
df_beat_by_district_neighborhood_name.shape

(277, 5)

In [10]:
df_beat_by_district_neighborhood_name[df_beat_by_district_neighborhood_name['beat_num'] == 3100]

,district,sector,beat,beat_num,district_name
1,31,0,0,3100,NaN
55,31,0,0,3100,NaN
263,31,0,0,3100,NaN


#### District 31 is not a data mistake;   the_geom MULTIPOLYGON column above had it listed in Antartica;   web search turned up a single brief mention on a Citizens Office of Police Accountability webpage  but server error upon click thru;  phone call to COPA number listed on that page went unreturned;   a couple phone calls around and it does exist but what it is or where(?) it's at got grudging mumbles and non-answers;   which is fine as I am a data scientist, EE, investor, trader, runner, cyclist, pizza eater, a Lincoln and Franklin biography nut, US citizen, taxpayer, and a bunch of other things etc etc etc, but not a news reporter trying to rip somebody a new hole;  as a data scientist  I checked that the data was sound and just left it  alone after that

#### Merge beat numbers, district names and numbers into main dataframe 

In [11]:
df = pd.merge(df, df_beat_by_district_neighborhood_name,\
                   how= 'left', left_on= 'Beat', right_on= 'beat_num')

In [12]:
df.head(1)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,district,sector,beat,beat_num,district_name
0,8926266,HV598372,2012-11-15 00:01:00,013XX W CORNELIA AVE,0810,THEFT,OVER $500,APARTMENT,False,False,1924,19.0,44.0,6.0,06,1166701.0,1923359.0,2012,02/10/2018 03:50:01 PM,41.945267,-87.662672,"(41.945267201, -87.662671562)",19.0,2.0,2.0,1924.0,Town Hall


#### Drop some unneeded columns and make column names uniform

In [13]:
df = df.drop(['X Coordinate', 'Y Coordinate'], axis= 1)

In [14]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location', 'district', 'sector', 'beat', 'beat_num', 'district_name'], dtype='object')

In [15]:
col_lower_caps_and_snake_case = ['id', 'case_number', 'date', 'block', 'iucr', 'primary_type', 'description',
                  'location_description', 'arrest', 'domestic', 'beat', 'district', 'ward',
                  'community_area', 'fbi_code', 'year', 'updated_on', 'latitude', 'longitude',
                  'location', 'district_x', 'sector', 'beat_x', 'beat_num', 'district_name']

In [16]:
df.columns = col_lower_caps_and_snake_case

In [17]:
df.head(1)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,year,updated_on,latitude,longitude,location,district_x,sector,beat_x,beat_num,district_name
0,8926266,HV598372,2012-11-15 00:01:00,013XX W CORNELIA AVE,0810,THEFT,OVER $500,APARTMENT,False,False,1924,19.0,44.0,6.0,06,2012,02/10/2018 03:50:01 PM,41.945267,-87.662672,"(41.945267201, -87.662671562)",19.0,2.0,2.0,1924.0,Town Hall


#### Save the cleaned and prepped dataframe

In [18]:
df.to_csv('../data/df.csv', index= False)